In [203]:
import pandas as pd
import time
import warnings
import requests

warnings.filterwarnings('ignore')

## Extracting Data

### Database Attandances

In [208]:
def get_data_gsheet(url=str, path=str, filename=str, sheetname=str):
    path = f'./{path}/{filename}.xlsx'
    
    try:
        os.remove(path)
    except:
        None
        
    output_filename = path
    
    # get the data from spreadsheet
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_filename, "wb") as f:
            f.write(response.content)

    # read data
    data = pd.read_excel(path, sheet_name=sheetname)
    return data

In [225]:
# database karyawan
db_emp = get_data_gsheet(
    url="https://docs.google.com/spreadsheets/d/18zheER6i6d72o9tJqFe93ljsETmiFUYYYr_N8-OKgNw/export?format.xlsx",
    path='Database',
    filename='database_karyawan',
    sheetname='Database Karyawan'
)

# cleaning data 
db_emp.columns = [''.join('_'.join(i.lower().split()).split('.')) for i in db_emp.columns.tolist()]
db_emp = db_emp.dropna(subset=['no_urut'])

# feature engineering
db_emp['id_2'].replace('37#RIO Office Equipment','37#RIO Utama', inplace=True)

### Database Fingerprint

In [186]:
def split_time_string(time_string=list):
    return [time_string[i:i+5] for i in range(0, len(time_string), 5)]

In [187]:
def time_category(data=str, date=str):
    data = pd.to_datetime(f"{date} {data}")
    day = data.day_name()

    if day != 'Friday':
        if pd.to_datetime(data) < pd.to_datetime(f'{date} 12:00'):
            msg = 'Cek In'
        elif pd.to_datetime(data) >= pd.to_datetime(f'{date} 12:00') and pd.to_datetime(data) < pd.to_datetime(f'{date} 16:30'):
            msg = 'Cek Rest'
        else:
            msg = 'Cek Out'
    else:
        if pd.to_datetime(data) < pd.to_datetime(f'{date} 11:00'):
            msg = 'Cek In'
        elif pd.to_datetime(data) >= pd.to_datetime(f'{date} 11:00') and pd.to_datetime(data) < pd.to_datetime(f'{date} 16:30'):
            msg = 'Cek Rest'
        else:
            msg = 'Cek Out'
    return msg

In [189]:
def split_time_category(data=list, date=str, condition=str):
    _ = []
    for i in data:
        n_time = i
        category_time = time_category(i, date)
        _.append(pd.DataFrame({'time':[n_time], 'category':[category_time]}))
    msg = pd.concat(_).reset_index(drop=True)
    
    try:
        if condition == 'Cek In':
            msg = msg[msg.category == condition]['time'].min()
        elif condition == 'Cek Rest Out':
            msg = msg[msg.category == 'Cek Rest']['time'].min()
        elif condition == 'Cek Rest In':
            msg = msg[msg.category == 'Cek Rest']['time'].max()
        else:
            msg = msg[msg.category == 'Cek Out']['time'].max()
    except:
        msg = 'Unknown'
    return msg

In [252]:
def get_data_excel(self=None, date=str, branch=str):
    data = pd.read_excel('./Datasets/datasets_presensi.xls', sheet_name='Lap. Log Absen')
    
    data = data[data.index >= 3].reset_index(drop=True)[['Lap. Detail Absensi','Unnamed: 1','Unnamed: 2']]
    data.columns = ['log_absen','unknown','id']
    data = data[['id','log_absen']]
    data['id'] = data['id'].ffill()
    data = data[data.log_absen != 'ID:']
    data['log_absen'].fillna('-', inplace=True)
    data = data[data.log_absen != '-'].reset_index(drop=True)
    data['log_absen'] = data.log_absen.apply(lambda x: split_time_string(str(x)))

    data['cek_in'] = data.log_absen.apply(lambda x: split_time_category(x, date=date, condition='Cek In'))
    data['cek_rest_out'] = data.log_absen.apply(lambda x: split_time_category(x, date=date, condition='Cek Rest Out'))
    data['cek_rest_in'] = data.log_absen.apply(lambda x: split_time_category(x, date=date, condition='Cek Rest In'))
    data['cek_out'] = data.log_absen.apply(lambda x: split_time_category(x, date=date, condition='Cek Out'))
    data.drop(columns=['log_absen'], inplace=True)
    data['branch'] = branch
    data['id_2'] = data['id'] + '#' + data['branch']

    data = data.merge(db_emp[db_emp.status_bekerja == 'Aktif'][['id_2','nama_karyawan_(sesuai_ktp)']], on='id_2', how='left').drop(columns=['id_2','branch'])
    data.columns = ['id','cek_in','cek_rest_out','cek_rest_in','cek_out','emp_name']
    data = data[['id','emp_name','cek_in','cek_rest_out','cek_rest_in','cek_out']]
    data = data.dropna(subset=['emp_name'])
    return data

In [257]:
get_data_excel(date='2025-06-20', branch='RIO Digital Printing')

,id,emp_name,cek_in,cek_rest_out,cek_rest_in,cek_out
0,1,Yance Wongkar,07:19,13:00,13:00,19:19
1,2,Christian Gosal,07:53,15:39,15:39,19:26
2,4,Ulul Azmi,07:55,14:01,14:23,18:02
3,75,Kalvin Sawatu,07:42,12:48,13:28,18:00
4,84,Forlan,07:47,13:39,14:40,18:04
5,25,Nurul Khasanah,07:32,12:31,12:31,21:04
6,92,Mohamad Adrian,07:21,12:06,12:55,18:07
7,94,Bima Ruseng,07:46,12:01,12:57,18:01
8,95,Tri Putri Rahma,07:47,13:27,14:16,18:44
9,34,Denny,07:43,11:36,13:32,18:16


## Transforming Data

## Loading Data